This notebook is to accompany the paper "CREATING A SCORE INDEPENDENT PERFORMANCE MEASURE FOR FOOTBALL MATCHES, USING NEURAL NETWORKS". It contains the code used to preprocess the data, create the datasets, train the models and evaluate them. The datasets and the models used in the paper can be loaded in this notebook. If the code to train the model is executed it may not produce the same model produced in the paper, due to the inherent stochasticity of the training algorithms used. The exact models produced and referenced in the paper can be loaded from file, and used to evaluate the data in the data set.

<h1>Libraries Used</h1>

The libraries used in this notebook are listed and imported below.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import model_from_json
from tensorflow.keras.callbacks import TensorBoard
import os
import datetime

<h1>Data Pre-processing </h1>

The data for this project was provided in several large JSON files. In order to use the data, it first had to be converted into the appropriate form. This only needed to be done once, and the pre-processed data could then be saved, in this case as CSV files. Due to the size of the raw data files, only the processed data is supplied with this notebook. The original JSON files can be downloaded [here](https://figshare.com/collections/Soccer_match_event_dataset/4415000/5). The code below will not run without these data files, but is provided for reference. The processed data can be loaded, however.

In [ ]:
matches_england=pd.read_json('matches/matches_england.json') #The Json files are loaded using Pandas.
matches_spain=pd.read_json('matches/matches_spain.json')
matches_germany=pd.read_json('matches/matches_germany.json')
matches_italy=pd.read_json('matches/matches_italy.json')
matches_france=pd.read_json('matches/matches_france.json')
matches = pd.concat([matches_england,matches_spain, matches_germany,matches_italy,matches_france], ignore_index=True) #The matches and events for different competitions are stored in separate files, and so must be concatenated together.
events_england = pd.read_json('events/events_england.json')
events_spain = pd.read_json('events/events_Spain.json')
events_Germany = pd.read_json('events/events_Germany.json')
events_Italy = pd.read_json('events/events_Italy.json')
events_france = pd.read_json('events/events_France.json')
events = pd.concat([events_england,events_spain,events_Germany,events_Italy,events_france],ignore_index=True)
detailedStats = events.subEventId.unique() #An array of the different event and subevent ID's is needed to build the statistics array for each match.
simpleStats = events.eventId.unique()
def getDetailedMatchStats(x): #This function, when given a match number, returns an array with the count for each type event from the detailed array. If this event did not occur in the match, the array will contain a 0 for that entry.
    hometeam,awayteam=getTeamIds(x) #The ID's for the teams in the match are provided here.
    matchId = matches.iloc[x]['wyId'] #Pandas is used to get the specific match ID, an internal identifier used within the data sets, as opposed to the match number, which is an index. 
    homeTeamStats = events[(events['matchId']==matchId)&(events['teamId'] == hometeam)].groupby('subEventId').count()['tags'] #Pandas allows for specific, efficient filtering of data sets. This code provides the 
    awayTeamStats = events[(events['matchId']==matchId)&(events['teamId'] == awayteam)].groupby('subEventId').count()['tags'] #counts of each sub event type for a given team in this match.
    matchStats = []
    for i in detailedStats: #The array of possible stats is iterated through and the count is placed in the array. 
        matchStats.append(homeTeamStats.get(i,0)) #The home team stats are provided first, then the away team.
        matchStats.append(awayTeamStats.get(i,0))
    return np.asarray(matchStats) #The array is returned as a numpy array.

def getSimpleMatchStats(x): #The exact same process can be done with the simple stats array, to provide statistics for the simple model.
    hometeam,awayteam=getTeamIds(x)
    matchId = matches.iloc[x]['wyId']
    homeTeamStats = events[(events['matchId']==matchId)&(events['teamId'] == hometeam)].groupby('eventId').count()['tags']
    awayTeamStats = events[(events['matchId']==matchId)&(events['teamId'] == awayteam)].groupby('eventId').count()['tags']
    matchStats = []
    for i in simpleStats:
        matchStats.append(homeTeamStats.get(i,0))
        matchStats.append(awayTeamStats.get(i,0))
    return np.asarray(matchStats)

def getTeamIds(x): #This function returns the Team IDs for the teams that took part in a given match.
    homeId, awayId = matches.iloc[x]['teamsData']
    return int(homeId),int(awayId)

def getScore(x): #This function gets the score for a given match, used to determine the class labels for each match.
    homeId,awayId = matches.iloc[x]['teamsData']
    homeId,awayId = int(homeId),int(awayId)
    h,a = matches.iloc[x]['teamsData'].values()
    return h['score'],a['score']

simpleStatsArray = []
detailedStatsArray = []
scores = []
for i in range(0,int(matches.shape[0])):
    simpleStatsArray.append(getSimpleMatchStats(i))
    detailedStatsArray.append(getDetailedMatchStats(i))
    scores.append(getScore(i))

scores = np.asarray(scores) 
homewin = scores[:,0]>scores[:,1] #A class label is determined by comparing the scores of the two teams.
draw = scores[:,0]==scores[:,1]
awaywin = scores[:,0]<scores[:,1] 
result = np.stack([homewin,draw,awaywin],axis=1) #The labesl are stacked into a single array.
result = result*1 #This is converted to a one hot array.


Once run, the above code provided 3 arrays, two different statistics arrays and the class label array. These were then saved to CSV files and can be loaded below.


In [3]:
simpleStatsArray = pd.read_csv("simpleStatsArray.csv")
simpleStatsArray = np.asarray(simpleStatsArray)
simpleStatsArray = simpleStatsArray[:,1:]
detailedStatsArray = pd.read_csv('detailedStatsArray.csv')
detailedStatsArray = np.asarray(detailedStatsArray)
detailedStatsArray = detailedStatsArray[:,1:]
results = pd.read_csv('results.csv')
results = np.asarray(results)
results = results[:,1:]


<h1>Building a Model</h1>
With the data loaded, we can build and train models. The code for building a model is below.

In [ ]:
def create_model(layer1, layer2, layer3): #The function takes 3 parameters, the number of nodes on each of the hidden layers.
    model = Sequential()
    model.add(Dense(layer1,input_dim=20, activation='relu')) #The input layer needs to match the dimensionality of the data set.
    model.add(Dense(layer2, activation = 'relu'))
    model.add(Dense(layer3, activation = 'relu'))
    model.add(Dense(3,activation='softmax')) #The output layer has 3 nodes, one for each class label. Softmax outputs a vector of categotical probabilities
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

The above code returns a model that can be fit to the data set, and validated against an unseen portion of it.
The models used in the paper have been provided and so can be loaded from a file, so it is unnecessary to refit a model. This code is included for completeness.


In [ ]:
model = create_model(8,8,8) #A model is created with given parameters.
log_dir = os.path.join('logs','fit',datetime.datetime.now().strftime('%Y%m%d-%H%M%S')) #This string is used to indentify the model in TensorBoard.
tensorboard_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)
history = model.fit(simpleStatsArray, results,validation_split = 0.2,epochs = 50, batch_size = 5, callbacks=[tensorboard_callback])

During the development process I trained 200 models for each data set and selected the one with the highest validation accuracy. These models can be loaded from the file below.

In [4]:
simpleClassifier = load_model('simple_classifier.h5')
detailedClassifier = load_model('detailed_classifier.h5')


These models achieved the highest accuracy and so were used to create the results detailed in the document. Below they are used to predict the outcome of all the matches in the data set.

<h1>Using the Models</h1>

In [5]:
simplePerformanceMeasure = simpleClassifier.predict(simpleStatsArray)
detailedPerformanceMeasure = detailedClassifier.predict(detailedStatsArray)

These 2 arrays contain the likelihood of each result, home win, draw or away win, for each of the matches in the data set. The results used in the paper are printed below.

In [6]:
indices = [35,33,55,854,938,1087,1215,467,506,1556,1595] #The indices of the matches used in the test set.
print(np.take(simplePerformanceMeasure,indices,axis=0))
print()
print(np.take(detailedPerformanceMeasure,indices,axis=0))

[[2.7984177e-04 9.9972016e-01 1.4970114e-09]
 [7.4925506e-01 6.5303335e-05 2.5067961e-01]
 [6.4440101e-02 2.7205525e-02 9.0835434e-01]
 [4.5432900e-03 3.4453182e-03 9.9201137e-01]
 [9.9792693e-08 4.6926198e-04 9.9953067e-01]
 [2.6539459e-15 1.0149865e-19 1.0000000e+00]
 [9.9998164e-01 2.4300002e-06 1.5938405e-05]
 [9.9999940e-01 6.5054746e-07 2.9664842e-14]
 [1.4606271e-08 3.2505763e-05 9.9996746e-01]
 [1.4052362e-02 2.4408686e-01 7.4186081e-01]
 [1.0000000e+00 6.8397127e-13 7.1703768e-17]]

[[2.4452115e-28 1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 6.8910593e-16 0.0000000e+00]
 [9.9999988e-01 1.4424349e-08 1.7084355e-07]
 [1.0546842e-06 9.9999893e-01 4.1038444e-32]
 [2.2733140e-08 9.9999559e-01 4.3534988e-06]
 [1.9548748e-15 4.7113371e-09 1.0000000e+00]
 [1.0000000e+00 1.9427962e-13 3.3124564e-13]
 [1.0000000e+00 3.0273557e-14 4.6374581e-26]
 [3.5002199e-07 9.9999881e-01 8.0431340e-07]
 [7.5528832e-25 4.7761464e-13 1.0000000e+00]
 [1.0000000e+00 1.0371099e-15 8.4952149e-12]]
